# Experiment Initialization

Here, the terms of the experiment are defined, such as the location of the files in S3 (bucket and folder name), and each of the video prefixes (everything before the file extension) that need to be tracked. 

Note that these videos should be similar-ish: while we can account for differences in mean intensities between videos, particle sizes should be approximately the same, and (slightly less important) particles should be moving at about the same order of magnitude speed. In this experiment, these videos were taken in 0.4% agarose gel at 100x magnification and 100.02 fps shutter speeds with nanoparticles of about 100nm in diameter.

In [1]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '08_06_19_MPT_age_dependence' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_sizes = ['40','100']
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
ages = ['P14', 'P21', 'P28', 'P35']
slices = 3
for age in ages:
    for size in NP_sizes:
        for slic in range(1, slices+1):
            for num in range(1, vids+1):
                to_track.append('{}_{}nm_s{}_v{}'.format(age, size, slic, num))

In [2]:
to_track

['P14_40nm_s1_v1',
 'P14_40nm_s1_v2',
 'P14_40nm_s1_v3',
 'P14_40nm_s1_v4',
 'P14_40nm_s1_v5',
 'P14_40nm_s2_v1',
 'P14_40nm_s2_v2',
 'P14_40nm_s2_v3',
 'P14_40nm_s2_v4',
 'P14_40nm_s2_v5',
 'P14_40nm_s3_v1',
 'P14_40nm_s3_v2',
 'P14_40nm_s3_v3',
 'P14_40nm_s3_v4',
 'P14_40nm_s3_v5',
 'P14_100nm_s1_v1',
 'P14_100nm_s1_v2',
 'P14_100nm_s1_v3',
 'P14_100nm_s1_v4',
 'P14_100nm_s1_v5',
 'P14_100nm_s2_v1',
 'P14_100nm_s2_v2',
 'P14_100nm_s2_v3',
 'P14_100nm_s2_v4',
 'P14_100nm_s2_v5',
 'P14_100nm_s3_v1',
 'P14_100nm_s3_v2',
 'P14_100nm_s3_v3',
 'P14_100nm_s3_v4',
 'P14_100nm_s3_v5',
 'P21_40nm_s1_v1',
 'P21_40nm_s1_v2',
 'P21_40nm_s1_v3',
 'P21_40nm_s1_v4',
 'P21_40nm_s1_v5',
 'P21_40nm_s2_v1',
 'P21_40nm_s2_v2',
 'P21_40nm_s2_v3',
 'P21_40nm_s2_v4',
 'P21_40nm_s2_v5',
 'P21_40nm_s3_v1',
 'P21_40nm_s3_v2',
 'P21_40nm_s3_v3',
 'P21_40nm_s3_v4',
 'P21_40nm_s3_v5',
 'P21_100nm_s1_v1',
 'P21_100nm_s1_v2',
 'P21_100nm_s1_v3',
 'P21_100nm_s1_v4',
 'P21_100nm_s1_v5',
 'P21_100nm_s2_v1',
 'P21_100n

In [4]:
to_track[0:30]

['P14_40nm_s1_v1',
 'P14_40nm_s1_v2',
 'P14_40nm_s1_v3',
 'P14_40nm_s1_v4',
 'P14_40nm_s1_v5',
 'P14_40nm_s2_v1',
 'P14_40nm_s2_v2',
 'P14_40nm_s2_v3',
 'P14_40nm_s2_v4',
 'P14_40nm_s2_v5',
 'P14_40nm_s3_v1',
 'P14_40nm_s3_v2',
 'P14_40nm_s3_v3',
 'P14_40nm_s3_v4',
 'P14_40nm_s3_v5',
 'P14_100nm_s1_v1',
 'P14_100nm_s1_v2',
 'P14_100nm_s1_v3',
 'P14_100nm_s1_v4',
 'P14_100nm_s1_v5',
 'P14_100nm_s2_v1',
 'P14_100nm_s2_v2',
 'P14_100nm_s2_v3',
 'P14_100nm_s2_v4',
 'P14_100nm_s2_v5',
 'P14_100nm_s3_v1',
 'P14_100nm_s3_v2',
 'P14_100nm_s3_v3',
 'P14_100nm_s3_v4',
 'P14_100nm_s3_v5']

The videos used with this analysis are fairly large (2048 x 2048 pixels and 651 frames), and in cases like this, the tracking algorithm can quickly eat up RAM. In this case, we chose to crop the videos to 512 x 512 images such that we can run our jobs on smaller EC2 instances with 16GB of RAM. 

Note that larger jobs can be made with user-defined functions such that splitting isn't necessary-- or perhaps an intermediate amount of memory that contains splitting, tracking, and msd calculation functions all performed on a single EC2 instance.

The compiled functions in the knotlets module require access to buckets on AWS. In this case, we will be using a publicly (read-only) bucket. If users want to run this notebook on their own, will have to transfer files from nancelab.publicfiles to their own bucket, as it requires writing to S3 buckets.

In [5]:
import diff_classifier.knotlets as kn

In [6]:
# This cell uses the function kn.split() to split all of the videos contained in 'to_track' into 16 smaller videos on which the actual tracking will be performed
for prefix in to_track[0:30]:
    kn.split(prefix, remote_folder=remote_folder, bucket=bucket)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v1_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v1_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v1_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v1_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ub

Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_0_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_0_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_0_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_0_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_1_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_1_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_1_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_1_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_2_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v1_2_1.tif
Done with 

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v2_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v2_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v2_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v2_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ub

Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_0_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_0_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_0_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_0_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_1_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_1_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_1_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_1_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_2_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_40nm_s1_v2_2_1.tif
Done with 

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v3_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v3_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v3_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_40nm_s1_v3_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ub

...

Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_0_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_0_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_0_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_0_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_1_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_1_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_1_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_1_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_2_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v3_2_1.tif


/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v4_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v4_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v4_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v4_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/hom

Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_0_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_0_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_0_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_0_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_1_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_1_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_1_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_1_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_2_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v4_2_1.tif


/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v5_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v5_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v5_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/P14_100nm_s3_v5_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/hom

Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_0_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_0_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_0_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_0_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_1_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_1_1.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_1_2.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_1_3.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_2_0.tif
Done with splitting. Should output file of name 08_06_19_MPT_age_dependence/P14_100nm_s3_v5_2_1.tif


## Tracking predictor

Tracking normally requires user input in the form of tracking parameters e.g. particle radius, linking max distance, max frame gap etc. When large datasets aren't required, each video can be manageably manually tracked using the TrackMate GUI. However, when datasets get large e.g. >20 videos, this can become extremely arduous. For videos that are fairly similar, you can get away with using similar tracking parameters across all videos. However, one parameter that is a little more noisy that the others is the quality filter value. Quality is a numerical value that approximate how likely a particle is to be "real." 

In this case, I built a predictor that estimates the quality filter value based on intensity distributions from the input images. Using a relatively small training dataset (5-20 videos), users can get fairly good estimates of quality filter values that can be used in parallelized tracking workflows.

Note: in the current setup, the predictor should be run in Python 3. While the code will run in Python 3, there are differences between the random number generators in Python2 and Python3 that I was not able to control for.

In [ ]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import diff_classifier.knotlets as kn
import numpy as np
from sklearn.externals import joblib

The regress_sys function should be run twice. When have_output is set to False, it generates a list of files that the user should manually track using Trackmate. Once the quality filter values are found, they can be used as input (y) to generate a regress object that can predict quality filter values for additional videos. Once y is assigned, set have_output to True and re-run the cell.

In [ ]:
tnum=20 #number of training datasets
pref = []
for num in to_track:                    
    for row in range(0, 4):
        for col in range(0, 4):
            pref.append("{}_{}_{}".format(num, row, col))

y = np.array([2.51, 1.01, 3.89, 3.89, 1.01, 0.81, 0.91, 1.59, 1.17, 1.01, 1.17, 1.16, 3.26, 2.06, 2.26, 1.41, 0.89, 2.09, 1.57, 2.39])

# Creates regression object based of training dataset composed of input images and manually
# calculated quality cutoffs from tracking with GUI interface.
regress = ij.regress_sys(remote_folder, pref, y, tnum, randselect=True,
                         have_output=True, bucket_name=bucket)
#Read up on how regress_sys works before running.

In [ ]:
#Pickle object
filename = 'regress.obj'
with open(filename,'wb') as fp:
    joblib.dump(regress,fp)

import boto3
s3 = boto3.client('s3')
aws.upload_s3(filename, remote_folder+'/'+filename, bucket_name=bucket)

Users should input all tracking parameters into the tparams object. Note that the quality value will be overwritten by values found using the quality predictor found above. Never change threshold, median intensity, or snr

In [ ]:
tparams1 = {'radius': 6.0, 'threshold': 0.0, 'do_median_filtering': False,
           'quality': 10.0, 'xdims': (0, 511), 'ydims': (1, 511),
           'median_intensity': 300.0, 'snr': 0.0, 'linking_max_distance': 15.0,
           'gap_closing_max_distance': 17.0, 'max_frame_gap': 8,
           'track_duration': 10.00}

## Cloudknot setup

Cloudknot requires the user to define a function that will be sent to multiple computers to run. In this case, the function knotlets.tracking will be used. We create a docker image that has the required installations (defined by the requirements.txt file from diff_classifier on Github, and the base Docker Image below that has Fiji pre-installed in the correct location.

Note that I modify the Docker image below such that the correct version of boto3 is installed. For some reason, versions later than 1.5.28 error out, so I specified 5.28 as the correct version. Run my_image.build below to double-check that the Docker image is successfully built prior to submitting the job to Cloudknot.

** Before you run this next cell, you have to switch the kernel from Python 3 to Python 2, by doing the following: **
 1. Kernel -> Restart and clear output
 2. Kernel -> Change Kernel -> Python 2
 3. Rerun cells required to run below cell
 
 ** One other important thing to note: 
 - If you are performing the tracking, be sure that the my_image =  line is set to ck.DockerImage(func=kn.tracking,...
 - If you are performing the MSD/feature calculation (after you've carried out the tracking), be sure that the my_image = line is se to ck.DockerImage(func=kn.assemble_msds, ...
     
     ** following the tracking, before you run assemble_msds, you need to run the cell below that redefines all_maps as all_maps2. all_maps2 doesn't include the tparams1 input, and allows the kn.assemble_msds section to run properly. It won't work with the tparams input. **
     
 Other than that, everything else should stay the same

In [ ]:
import cloudknot as ck
import os.path as op

github_installs=('https://github.com/ccurtis7/diff_classifier.git')
#my_image = ck.DockerImage(func=kn.tracking, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)
my_image = ck.DockerImage(func=kn.assemble_msds, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)
docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-5]+'5.28'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

Following the execution of this cell, you have to check that the requirements.txt file has the first line 'boto3==1.5.28'.
    - This file can be found in source -> diff-classifier -> notebooks -> development -> most recent file

If it doesn't, you may have to change the line in the cell above that says 'new_req = reg_string[0:req_string.find('\n')-4]+'5.28'+ reg_string[reg_string.find('\n'):]'

In [ ]:
my_image.build("ChABC_slice_2", image_name="test_image")

In [ ]:
to_track

The object all_maps is an iterable containing all the inputs sent to Cloudknot. This is useful, because if the user needs to modify some of the tracking parameters for a single video, this can be done prior to submission to Cloudknot.

In [ ]:
names = []
all_maps = []
for prefix in to_track:    
    for i in range(0, 4):
        for j in range(0, 4):
            names.append('{}_{}_{}'.format(prefix, i, j))
            all_maps.append(('{}_{}_{}'.format(prefix, i, j), remote_folder, bucket, 'regress.obj', 4, 4, (512, 512), tparams1))

In [ ]:
start_knot = 98

The Cloudknot knot object sets up the compute environment which will run the code. Note that the name must be unique. Every time you submit a new knot, you should change the name. I do this with the variable start_knot, which I vary for each run.

If larger jobs are anticipated, users can adjust both RAM and storage with the memory and image_id variables. Memory specifies the amount of RAM to be used. Users can build a customized AMI with as much space as they need, and enter the ID into image_ID. Read the Cloudknot documentation for more details.

In [ ]:
knot = ck.Knot(name='{}_b{}'.format('mike', start_knot),
               docker_image = my_image,
               memory = 16000,
               resource_type = "SPOT",
               bid_percentage = 100,
               #image_id = 'ami-0e00afdf500081a0d', #May need to change this line
               pars_policies=('AmazonS3FullAccess',))

result_futures = knot.map(all_maps2, starmap=True)

In [ ]:
knot.clobber()

You can track the progression of your run using the AWS Batch service online -- make sure you are looking at the right US Region.

After the run, you might have some that fail. This usually happens when the computers get claimed by someone paying more money, and your job gets booted from the aws computers. Because of this, you will need to start a knw cloudknot knot and rerun those vids. The set up for that is shown below.

Remember to clobber your knot!

In [ ]:
ck.get_region()

This creates a new all_maps2 array for any of the videos that failed to get analyzed the first time through. Double check that it worked well by printing the length of it immediately afterwards.

In [ ]:
missing = []
all_maps2 = []
import boto3
import botocore

s3 = boto3.resource('s3')


for name in names:
    try:
        s3.Object(bucket, '{}/Traj_{}.csv'.format(remote_folder, name)).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            missing.append(name)
            all_maps2.append((name, remote_folder, bucket, 'regress.obj',
                             4, 4, (512, 512), tparams1))
        else:
            print('Something else has gone wrong')

In [ ]:
print(len(all_maps2))

Make sure you change the name of your knot, either by changing the 'mike1' part or start_knot value.

In [ ]:
all_maps2 = []
for prefix in to_track:
    all_maps2.append((prefix, remote_folder, bucket, (512, 512), 651, 4, 4))

Users can monitor the progress of their job in the Batch interface. Once the code is complete, users should clobber their knot to make sure that all AWS resources are removed.

In [ ]:
to_track

## Downstream analysis and visualization

The knotlet.assemble_msds function (which can also potentially be submitted to Cloudknot as well for large jobs) calculates the mean squared displacements and trajectory features from the raw trajectory csv files found from the Cloudknot submission. It accesses them from the S3 bucket to which they were saved.

In [ ]:
for prefix in to_track:
    kn.assemble_msds(prefix, remote_folder, bucket='mckenna.data')
    print('Successfully output msds for {}'.format(prefix))

In [ ]:
for prefix in to_track[5:7]:
    kn.assemble_msds(prefix, remote_folder, bucket='ccurtis.data')
    print('Successfully output msds for {}'.format(prefix))

Diff_classifier includes some useful imaging tools as well, including checking trajectories, plotting heatmaps of trajectory features, distributions of diffusion coefficients, and MSD plots.

In [ ]:
import diff_classifier.heatmaps as hm
import diff_classifier.aws as aws

In [ ]:
to_track

In [ ]:
for vids in to_track:
    prefix = vids
    msds = 'msd_{}.csv'.format(prefix)
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3('{}/{}'.format(remote_folder, msds), msds, bucket_name=bucket)
    aws.download_s3('{}/{}'.format(remote_folder, feat), feat, bucket_name=bucket)
    hm.plot_trajectories(prefix, remote_folder=remote_folder, upload=True, figsize=(8, 8), bucket = bucket)
    print('Successfully uploaded trajectory plot for {}'.format(prefix))
    geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=4, y_range=0.5, umppx=0.07, fps=33, upload=True, remote_folder=remote_folder, bucket = bucket)
    aws.upload_s3('./geomean_{}.csv'.format(prefix), remote_folder+'/geomean_{}.csv'.format(prefix), bucket_name = bucket)
    aws.upload_s3('./geoSEM_{}.csv'.format(prefix), remote_folder+'/geoSEM_{}.csv'.format(prefix), bucket_name = bucket)
    aws.upload_s3('./msds_{}.png'.format(prefix), remote_folder+'/msds_{}.png'.format(prefix), bucket_name = bucket)
    print('Successfully uploaded csv files for {}'.format(prefix))
    
    

In [ ]:
geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=4, y_range=0.5, umppx=0.07, fps=100, upload=True, remote_folder=remote_folder, bucket = bucket)
aws.upload_s3('./geomean_{}.csv'.format(prefix), remote_folder+'/geomean_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./geoSEM_{}.csv'.format(prefix), remote_folder+'/geoSEM_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./msds_{}.png'.format(prefix), remote_folder+'/msds_{}.png'.format(prefix), bucket_name = bucket)

In [ ]:
hm.plot_heatmap(prefix, upload=False)

In [ ]:
hm.plot_particles_in_frame(prefix, y_range=500, upload=False)

In [ ]:
prefix = to_track[0]

msds = 'msd_{}.csv'.format(prefix)
feat = 'features_{}.csv'.format(prefix)
aws.download_s3('{}/{}'.format(remote_folder, msds), msds, bucket_name=bucket)
aws.download_s3('{}/{}'.format(remote_folder, feat), feat, bucket_name=bucket)

In [ ]:
hm.plot_trajectories(prefix, remote_folder=remote_folder, upload=True, figsize=(8, 8), bucket = bucket)

In [ ]:
geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=4, y_range=0.5, umppx=0.07, fps=100, upload=True, remote_folder=remote_folder, bucket = bucket)
aws.upload_s3('./geomean_{}.csv'.format(prefix), remote_folder+'/geomean_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./geoSEM_{}.csv'.format(prefix), remote_folder+'/geoSEM_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./msds_{}.png'.format(prefix), remote_folder+'/msds_{}.png'.format(prefix), bucket_name = bucket)

## Converting msd files

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

In [ ]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '07_10_19_MPT_HA_solutions_trial2' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_size = '40'
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
mol_weights = ['high', 'low', 'med']

for MW in mol_weights:
    for num in range(1, vids+1):
        to_track.append('{}MW_'.format(MW)+NP_size+'nm_vid_{}'.format(num))

In [ ]:
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

for prefix in to_track:
    filename = 'geomean_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+filename, filename, bucket_name=bucket)
    local_name = filename
    merged = pd.read_csv(local_name)
    merged.columns = ['log']
    merged['exp'] = 0
    for rows in range(0,len(merged)):
        log_value = merged['log'].iloc[rows]
        exp_value = math.exp(log_value)
        merged.loc[rows,'exp'] = exp_value
    merged.to_csv('adj_'+filename, mode='w', index = False)
    aws.upload_s3('./adj_'+filename, remote_folder+'/adj_'+filename, bucket_name = bucket)

In [ ]:
pref = 'adj_geomean_'

mol_wts = ['high', 'low', 'med']
num_vids = 5

for MW in mol_wts:
    avg_df = pd.DataFrame()
    for vid in range(1, num_vids+1):
        exp = pd.read_csv(pref+'{}MW_40nm_vid_{}.csv'.format(MW, str(vid)))
        avg_df = pd.concat([avg_df, exp['exp']], axis=1)
    avg_df = avg_df.mean(axis = 1)
    avg_df.to_csv('avg_'+pref+'{}MW_40nm.csv'.format(MW), mode='w', index = False)
    aws.upload_s3('./avg_'+pref+'{}MW_40nm.csv'.format(MW), remote_folder+'/avg_'+pref+'{}MW_40nm.csv'.format(MW), bucket_name=bucket)
    

## Generating Histograms

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import os
import os.path as op
from os import listdir

In [ ]:
to_track

In [ ]:
high_track = to_track[0:5]
low_track = to_track[5:10]
med_track = to_track[10:15]

print(high_track)
print(low_track)
print(med_track)

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

high_track = to_track[0:5]
low_track = to_track[5:10]
med_track = to_track[10:15]

low_MW = pd.DataFrame()
med_MW = pd.DataFrame()
high_MW = pd.DataFrame()

um_px = 0.07
fps = 33

for prefix in low_track:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, 'trial_2_'+feat, bucket_name=bucket)
    merged = pd.read_csv('trial_2_'+feat)
    low_MW = pd.concat([low_MW, merged['Deff1']*um_px*um_px*fps/10], axis=0)
    
for prefix in med_track:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, 'trial_2_'+feat, bucket_name=bucket)
    merged = pd.read_csv('trial_2_'+feat)
    med_MW = pd.concat([med_MW, merged['Deff1']*um_px*um_px*fps/10], axis=0)
    
for prefix in high_track:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, 'trial_2_'+feat, bucket_name=bucket)
    merged = pd.read_csv('trial_2_'+feat)
    high_MW = pd.concat([high_MW, merged['Deff1']*um_px*um_px*fps/10], axis=0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

low_no_nan = low_MW.replace(0,np.nan)
med_no_nan = med_MW.replace(0,np.nan)
high_no_nan = high_MW.replace(0,np.nan)


log_Deff_low = np.log(low_no_nan[0].dropna())
#print(log_Deff_low)
log_Deff_med = np.log(med_no_nan[0].dropna())
#print(log_Deff_low)
log_Deff_high = np.log(high_no_nan[0].dropna())

test_bins = np.linspace(-12, 0, 76)

low_hist, low_bins = np.histogram(log_Deff_low, bins=test_bins)
med_hist, med_bins = np.histogram(log_Deff_med, bins=test_bins)
high_hist, high_bins = np.histogram(log_Deff_high, bins=test_bins)

low_avg = np.mean(log_Deff_low)
med_avg = np.mean(log_Deff_med)
high_avg = np.mean(log_Deff_high)

plt.rc('axes', linewidth=2)
low_plot, med_plot, high_plot = low_hist, med_hist, high_hist
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, high_plot, color='royalblue', align='center', width=width, alpha=0.6, label='high MW')
plt.axvline(high_avg, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, med_plot, color='darkgreen', align='center', width=width, alpha=0.7, label='medium MW')
plt.axvline(med_avg, color='darkgreen', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, low_plot, color='firebrick', align='center', width=width, alpha=1, label='low MW')
plt.axvline(low_avg, color='firebrick', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,600))
plt.legend(fontsize='large', loc=2)
plt.show()

In [ ]:
print(low_avg, med_avg, high_avg)
print(np.exp(low_avg), np.exp(med_avg), np.exp(high_avg))

## Combined HA Solution Histograms

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import os
import os.path as op
from os import listdir

In [ ]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '06_27_19_MPT_HA_solutions' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_sizes = ['40', '100']
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
mol_weights = ['high', 'low', 'med']

for size in NP_sizes:
    for MW in mol_weights:
        for num in range(1, vids+1):
            to_track.append('{}MW_'.format(MW)+'{}nm_vid_{}_{}MW_'.format(size, num, MW)+'{}nm_vid_{}'.format(size,num))
            
    

In [ ]:
print(len(to_track))
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

high_track_40 = to_track[0:5]
low_track_40 = to_track[5:10]
med_track_40 = to_track[10:15]
high_track_100 = to_track[15:20]
low_track_100 = to_track[20:25]
med_track_100 = to_track[25:30]

low_MW_40 = pd.DataFrame()
med_MW_40 = pd.DataFrame()
high_MW_40 = pd.DataFrame()
low_MW_100 = pd.DataFrame()
med_MW_100 = pd.DataFrame()
high_MW_100 = pd.DataFrame()

um_px = 0.07
fps_40 = 33
fps_100 = 100

for prefix in low_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    low_MW_40 = pd.concat([low_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
low_MW_40.to_csv('40nm_lowMW_trial_1.csv', mode='w', index = False)
    
for prefix in med_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    med_MW_40 = pd.concat([med_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
med_MW_40.to_csv('40nm_medMW_trial_1.csv', mode='w', index = False)
    
for prefix in high_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    high_MW_40 = pd.concat([high_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
high_MW_40.to_csv('40nm_highMW_trial_1.csv', mode='w', index = False)    
    
for prefix in low_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    low_MW_100 = pd.concat([low_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
low_MW_100.to_csv('100nm_lowMW_trial_1.csv', mode='w', index = False)
    
for prefix in med_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    med_MW_100 = pd.concat([med_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
med_MW_100.to_csv('100nm_medMW_trial_1.csv', mode='w', index = False)
    
for prefix in high_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    high_MW_100 = pd.concat([high_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
high_MW_100.to_csv('100nm_highMW_trial_1.csv', mode='w', index = False)

In [ ]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '07_10_19_MPT_HA_solutions_trial2' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_sizes = ['40', '100']
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
mol_weights = ['high', 'low', 'med']

for MW in mol_weights:
    for size in NP_sizes:
        for num in range(1, vids+1):
            to_track.append('{}MW_{}nm_vid_{}'.format(MW, size, num))
            


In [ ]:
print(len(to_track))
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

high_track_40 = to_track[0:5]
low_track_40 = to_track[10:15]
med_track_40 = to_track[20:25]
high_track_100 = to_track[5:10]
low_track_100 = to_track[15:20]
med_track_100 = to_track[25:30]


low_MW_40 = pd.DataFrame()
med_MW_40 = pd.DataFrame()
high_MW_40 = pd.DataFrame()
low_MW_100 = pd.DataFrame()
med_MW_100 = pd.DataFrame()
high_MW_100 = pd.DataFrame()

um_px = 0.07
fps_40 = 33
fps_100 = 100

for prefix in low_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    low_MW_40 = pd.concat([low_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
low_MW_40.to_csv('40nm_lowMW_trial_2.csv', mode='w', index = False)
    
for prefix in med_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    med_MW_40 = pd.concat([med_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
med_MW_40.to_csv('40nm_medMW_trial_2.csv', mode='w', index = False)
    
for prefix in high_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    high_MW_40 = pd.concat([high_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
high_MW_40.to_csv('40nm_highMW_trial_2.csv', mode='w', index = False)    
    
for prefix in low_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    low_MW_100 = pd.concat([low_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
low_MW_100.to_csv('100nm_lowMW_trial_2.csv', mode='w', index = False)
    
for prefix in med_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    med_MW_100 = pd.concat([med_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
med_MW_100.to_csv('100nm_medMW_trial_2.csv', mode='w', index = False)
    
for prefix in high_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    high_MW_100 = pd.concat([high_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
high_MW_100.to_csv('100nm_highMW_trial_2.csv', mode='w', index = False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

lowMW_40nm_1 = pd.read_csv('40nm_lowMW_trial_1.csv')
medMW_40nm_1 = pd.read_csv('40nm_medMW_trial_1.csv')
highMW_40nm_1 = pd.read_csv('40nm_highMW_trial_1.csv')

lowMW_40nm_2 = pd.read_csv('40nm_lowMW_trial_2.csv')
medMW_40nm_2 = pd.read_csv('40nm_medMW_trial_2.csv')
highMW_40nm_2 = pd.read_csv('40nm_highMW_trial_2.csv')

lowMW_40nm_Deffs = pd.DataFrame()
medMW_40nm_Deffs = pd.DataFrame()
highMW_40nm_Deffs = pd.DataFrame()

lowMW_40nm_Deffs = pd.concat([lowMW_40nm_1, lowMW_40nm_2], axis=0)
medMW_40nm_Deffs = pd.concat([medMW_40nm_1, medMW_40nm_2], axis=0)
highMW_40nm_Deffs = pd.concat([highMW_40nm_1, highMW_40nm_2], axis=0)

low_40_no_nan = lowMW_40nm_Deffs.replace(0,np.nan)
med_40_no_nan = medMW_40nm_Deffs.replace(0,np.nan)
high_40_no_nan = highMW_40nm_Deffs.replace(0,np.nan)

log_Deff_low_40 = np.log(low_40_no_nan.dropna())
log_Deff_med_40 = np.log(med_40_no_nan.dropna())
log_Deff_high_40 = np.log(high_40_no_nan.dropna())

test_bins = np.linspace(-12, 0, 76)

low_hist_40, low_bins_40 = np.histogram(log_Deff_low_40, bins=test_bins)
med_hist_40, med_bins_40 = np.histogram(log_Deff_med_40, bins=test_bins)
high_hist_40, high_bins_40 = np.histogram(log_Deff_high_40, bins=test_bins)

low_avg_40 = np.mean(log_Deff_low_40)[0]
med_avg_40 = np.mean(log_Deff_med_40)[0]
high_avg_40 = np.mean(log_Deff_high_40)[0]

plt.rc('axes', linewidth=2)
low_plot_40, med_plot_40, high_plot_40 = low_hist_40, med_hist_40, high_hist_40
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, high_plot_40, color='royalblue', align='center', width=width, alpha=0.6, label='40nm high MW')
plt.axvline(high_avg_40, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, med_plot_40, color='darkgreen', align='center', width=width, alpha=0.7, label='40nm medium MW')
plt.axvline(med_avg_40, color='darkgreen', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, low_plot_40, color='firebrick', align='center', width=width, alpha=1, label='40nm low MW')
plt.axvline(low_avg_40, color='firebrick', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,1400))
plt.legend()
plt.show()

In [ ]:
print(low_avg_40, med_avg_40, high_avg_40)
print(np.exp(low_avg_40), np.exp(med_avg_40), np.exp(high_avg_40))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

lowMW_100nm_1 = pd.read_csv('100nm_lowMW_trial_1.csv')
medMW_100nm_1 = pd.read_csv('100nm_medMW_trial_1.csv')
highMW_100nm_1 = pd.read_csv('100nm_highMW_trial_1.csv')

lowMW_100nm_2 = pd.read_csv('100nm_lowMW_trial_2.csv')
medMW_100nm_2 = pd.read_csv('100nm_medMW_trial_2.csv')
highMW_100nm_2 = pd.read_csv('100nm_highMW_trial_2.csv')

lowMW_100nm_Deffs = pd.DataFrame()
medMW_100nm_Deffs = pd.DataFrame()
highMW_100nm_Deffs = pd.DataFrame()

lowMW_100nm_Deffs = pd.concat([lowMW_100nm_1, lowMW_100nm_2], axis=0)
medMW_100nm_Deffs = pd.concat([medMW_100nm_1, medMW_100nm_2], axis=0)
highMW_100nm_Deffs = pd.concat([highMW_100nm_1, highMW_100nm_2], axis=0)

low_100_no_nan = lowMW_100nm_Deffs.replace(0,np.nan)
med_100_no_nan = medMW_100nm_Deffs.replace(0,np.nan)
high_100_no_nan = highMW_100nm_Deffs.replace(0,np.nan)

log_Deff_low_100 = np.log(low_100_no_nan.dropna())
log_Deff_med_100 = np.log(med_100_no_nan.dropna())
log_Deff_high_100 = np.log(high_100_no_nan.dropna())

test_bins = np.linspace(-12, 0, 76)

low_hist_100, low_bins_100 = np.histogram(log_Deff_low_100, bins=test_bins)
med_hist_100, med_bins_100 = np.histogram(log_Deff_med_100, bins=test_bins)
high_hist_100, high_bins_100 = np.histogram(log_Deff_high_100, bins=test_bins)

low_avg_100 = np.mean(log_Deff_low_100)[0]
med_avg_100 = np.mean(log_Deff_med_100)[0]
high_avg_100 = np.mean(log_Deff_high_100)[0]

plt.rc('axes', linewidth=2)
low_plot_100, med_plot_100, high_plot_100 = low_hist_100, med_hist_100, high_hist_100
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, high_plot_100, color='royalblue', align='center', width=width, alpha=0.6, label='100nm high MW')
plt.axvline(high_avg_100, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, med_plot_100, color='darkgreen', align='center', width=width, alpha=0.7, label='100nm medium MW')
plt.axvline(med_avg_100, color='darkgreen', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, low_plot_100, color='firebrick', align='center', width=width, alpha=1, label='100nm low MW')
plt.axvline(low_avg_100, color='firebrick', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,600))
plt.legend()
plt.show()

In [ ]:
print(low_avg_100, med_avg_100, high_avg_100)
print(np.exp(low_avg_100), np.exp(med_avg_100), np.exp(high_avg_100))